In [3]:
n=3

In [4]:
G = SymmetricGroup(n)

In [5]:
SGA = G.algebra(QQ)

In [6]:
A = SGA.dft(); A

[   1    1    1    1    1    1]
[   1 -1/2 -1/2  1/2  1/2   -1]
[   0 -3/4  3/4  3/4 -3/4    0]
[   0    1   -1    1   -1    0]
[   1 -1/2 -1/2 -1/2 -1/2    1]
[   1    1    1   -1   -1   -1]

In [7]:
#is the matrix unitary?
#A*A^T, the Gram matrix, is diagonal, but not the identity. so A is not unitary.

In [8]:
print((A*A.transpose()).str())

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]


In [9]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick:
#OPTION 1: define a new invariant inner product <x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
#compute an orthonormal basis w.r.t. this inner product by computing the Gram matrix A and using A.gram_schmidt()
#OPTION 2: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2

In [10]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,unitary=False):
    n = sum(partition)
    G = SymmetricGroup(n)
    specht_module = G.algebra(QQ).specht_module(partition)
    rho = specht_module.representation_matrix
    d_part = specht_module.dimension()
    if unitary:
        M = unitary_change_of_basis(partition)
        return sqrt(d_part/G.order())*sum(f(g)*M.inverse()*rho(g)*M for g in G)
    else:
        return sum(f(g)*rho(g) for g in G)

In [11]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
def delta(s):
    return lambda t: 1 if t == s else 0

In [12]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(n):
    return matrix([flatten([hat(delta(g),partition,unitary=True).list() for partition in Partitions(n)]) for g in G]).transpose()

In [13]:
#find the change-of-basis matrix M for which A = M*GS where GS is the Gram-Schmidt orthonormal basis of A
def unitary_change_of_basis(partition):
    specht_module = G.algebra(QQ).specht_module(partition)
    rho = specht_module.representation_matrix
    P = (1/G.order())*sum(rho(g)*rho(g).conjugate().transpose() for g in G)
    Q = P.principal_square_root()
    return Q

In [15]:
partition = Partitions(n)[1]

In [21]:
specht_module = G.algebra(QQ).specht_module(partition)

In [22]:
rho = specht_module.representation_matrix

In [23]:
d_part = specht_module.dimension()

In [24]:
Q = unitary_change_of_basis(partition); Q

[1/2*sqrt(2) + 1/2*sqrt(2/3) 1/2*sqrt(2) - 1/2*sqrt(2/3)]
[1/2*sqrt(2) - 1/2*sqrt(2/3) 1/2*sqrt(2) + 1/2*sqrt(2/3)]

In [25]:
U = (Q.inverse()*rho(G[5])*Q).simplify_full(); (U*U.transpose()).simplify_full()

[1 0]
[0 1]

In [ ]:
#check that the DFT is unitary
(unitary_dft(3)*unitary_dft(3).transpose()).simplify_full() == identity_matrix(factorial(3))

In [ ]:
eigs = matrix(CDF,unitary_dft(3).simplify_full()).eigenvalues()

In [ ]:
eigs

In [ ]:
#what are the eigenvalues?
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
eigs = A.eigenvalues(); eigs

In [ ]:
[abs(eig) for eig in eigs]

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))